In [1]:
import aocd
import re
import numpy as np
import itertools
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2024, day=12)
data = puzzle.input_data

In [2]:
grid = Grid(data, str)

In [3]:
# Directions and their "left-handed" check direction
directions = {
    (1, 0): (0,1),
    (-1,0): (0,-1),
    (0, 1): (-1,0),
    (0,-1): (1,0)
}

def get_num_sides(grid, point):
    s = 0
    for d in directions:
        # Check that there is actually an edge
        new_point = (point[0] + d[0], point[1] + d[1])
        if grid.in_bounds(new_point) and grid[new_point] == grid[point]:
            continue
            
        # If the piece next to you is out of bounds or different you are the end
        other_piece = (point[0] + directions[d][0],
                         point[1] + directions[d][1])
        if not grid.in_bounds(other_piece) or grid[other_piece] != grid[point]:
            s += 1
            continue
        check_point = (point[0] + d[0] + directions[d][0],
                         point[1] + d[1] + directions[d][1])
        # If both your opposite and your neighbors opposite are out of bounds
        # you are not the end
        if not grid.in_bounds(new_point) and not grid.in_bounds(check_point):
            continue
        # If your neighbor's opposite is the same as you then you are the end
        if grid[check_point] == grid[point]:
            s += 1
            continue
    return s
    

def fill(grid, point, point_set):
    point_set.append(point)
    sides = grid.sides(point)
    sides = [s for s in sides if grid[s] == grid[point]]
    num_fences = 4-len(sides)
    area = 1
    num_sides = get_num_sides(grid, point)
    for side in sides:
        if side in point_set:
            continue
        nf, ns, a, _ = fill(grid, side, point_set)
        num_fences += nf
        num_sides += ns
        area += a
    return num_fences, num_sides, area, point_set
    

In [4]:
possible_points = set(itertools.product(range(len(grid)), range(len(grid[0]))))
score_a = 0
score_b = 0
while(len(possible_points) > 0):
    nf, ns, a, p = fill(grid, list(possible_points)[0], [])
    possible_points -= set(p)
    score_a += nf*a
    score_b += ns*a

In [5]:
puzzle.answer_a = score_a

In [6]:
puzzle.answer_b = score_b